In [ ]:
import tools

def name_extraction_prompt_maker(single_value):
    prompts = [
        {
            "role": "user",
            "content": '''Please execute the following steps to extract all proper nouns from the input text:

Step 1: Identification

    Identify all proper nouns that are specific to the field of computer science.

Inclusion Criteria (Rule1):

    Only include proper nouns that are directly mentioned in the text.
    Focus on specific names related to:
        Malware
        Unique vulnerabilities (specifically referenced by CVE identifiers)
        Distinct network identifiers (IP addresses, domain names, URLs)
        Detailed Indicators of Compromise (IOCs) such as file paths, hash values, port numbers, registry key changes, MAC addresses, and unique signatures.
        Software names (e.g., antivirus programs, hacking tools, operating systems)
        Hardware names (e.g., specific models of routers, servers, or IoT devices)
        Specific names of hacker groups or threat actors
        Specific names of security protocols or encryption algorithms
        Specific names of programming languages, libraries, or frameworks
        Specific names of computer science concepts or technologies

Step 2: Exclusion
    After listing all names according to Rule1, evaluate each against Rule2.
    For each name you exclude based on Rule2, provide a concise justification. Rule2 states that you should exclude the following types of proper nouns:
        Human names
        Country names
        City names

Final Step: Final Output

    After the evaluation, present the final, curated list that adheres to Rule1 and Rule2.

    Format request:
    You should use the JSON format to present the names in each step.
    An example JSON output should look like this. Make sure the keys are the same as "Step 1", "Step 2", "Final Step", and the values are lists of names with double quotes.
    {
        "Step 1": ["ExampleMalware1", "ExampleMalware2", "Country1", "human1"],
        "Step 2": ["ExampleMalware1:Include", "ExampleMalware2:Include", "Country1:Exclude,country name", "human1:Exclude, human name"],
        "Final Step": ["ExampleMalware1", "ExampleMalware2"]
    }
            '''
        },
        {
            "role": "assistant",
            "content": "Understood. I will now proceed to extract the names."
        },
        {
            "role": "user",
            "content": "The malware named DarkComet was used by the hacker group APT29. It exploited the CVE-2012-0158 vulnerability to infiltrate systems using IP address 192.168.0.10. The attack was identified through the SHA256 hash: a3d5f8e7e6b8f2348d7b9d8a6f9a0f2a2c1e1f3f8a9f4f8d6c7f8e8f1f1a3d6b. The affected systems were running the Windows 7 operating system and utilized the D-Link DIR-615 router. The malware communicated using the SSL/TLS protocol and was written in C++. This incident was reported in New York, USA, and highlighted vulnerabilities in network security architecture. The person who discovered it was named John Doe."
        },
        {
            "role": "assistant",
            "content": '''```json{
                "Step 1": [
                    "DarkComet",
                    "APT29",
                    "CVE-2012-0158",
                    "192.168.0.10",
                    "SHA256:a3d5f8e7e6b8f2348d7b9d8a6f9a0f2a2c1e1f3f8a9f4f8d6c7f8e8f1f1a3d6b",
                    "Windows 7",
                    "D-Link DIR-615",
                    "SSL/TLS",
                    "C++",
                    "New York",
                    "USA",
                    "John Doe",
                    "network security architecture"
                ],
                "Step 2": [
                    "DarkComet:Include",
                    "APT29:Include",
                    "CVE-2012-0158:Include",
                    "192.168.0.10:Include",
                    "SHA256:a3d5f8e7e6b8f2348d7b9d8a6f9a0f2a2c1e1f3f8a9f4f8d6c7f8e8f1f1a3d6b:Include",
                    "Windows 7:Include",
                    "D-Link DIR-615:Include",
                    "SSL/TLS:Include",
                    "C++:Include",
                    "New York:Exclude, city name",
                    "USA:Exclude, country name",
                    "John Doe:Exclude, human name",
                    "network security architecture:Include"
                ],
                "Final Step": [
                    "DarkComet",
                    "APT29",
                    "CVE-2012-0158",
                    "192.168.0.10",
                    "SHA256:a3d5f8e7e6b8f2348d7b9d8a6f9a0f2a2c1e1f3f8a9f4f8d6c7f8e8f1f1a3d6b",
                    "Windows 7",
                    "D-Link DIR-615",
                    "SSL/TLS",
                    "C++",
                    "network security architecture"
                ]
            }```'''
        },
        {
            "role": "user",
            "content": f"Good job! Now, I will give you another new sentence. Follow the same steps to extract the names and provide the JSON format output. Don't use the example names from the previous example. Here is the sentence: \"{single_value}\""
        }
    ]
    return prompts

# Example usage
input_list = CoreDF["content part"].tolist()
input_prompts_batch = []
for i in input_list:
    input_prompts_batch.append(name_extraction_prompt_maker(i))
input_prompts_batch = input_prompts_batch * 3

code_name = "name extraction"

# Generate JSONL file object and file path
jsonl_file = tools.create_jsonl(input_prompts_batch, model='gpt4', temp=1, token=16384, jsonlname=code_name, possible_output=300)

# Upload JSONL file and execute batch process
ids = tools.upload_RUN_PAY_jsonl(jsonl_file, code_name)

# Get answers from the batch process
ans = tools.auto_down_ans(ids['batch_id'])

# Convert answers to DataFrame and add extracted keywords as a new column
import sys, pickle, os, json, re, time, random, logging, pandas as pd, numpy as np, matplotlib.pyplot as plt, scipy, sklearn, networkx as nx, importlib; sys.path.append('./Dropbox'); import tools; importlib.reload(tools)

df = tools.ans_to_df(ans)

from collections import Counter
import numpy as np

all_keep_items = []
total_correct = 0
partial_nan_count = 0
all_nan_count = 0

for i in range(len(input_list)):
    try:
        # Retrieve values for each step and handle NaN
        step1 = df.loc[i, 'Final Step']
        step2 = df.loc[i + len(input_list), 'Final Step']
        step3 = df.loc[i + 2 * len(input_list), 'Final Step']

        # Check if each step is NaN
        step1_is_nan = isinstance(step1, float) and np.isnan(step1)
        step2_is_nan = isinstance(step2, float) and np.isnan(step2)
        step3_is_nan = isinstance(step3, float) and np.isnan(step3)

        nan_count = sum([step1_is_nan, step2_is_nan, step3_is_nan])

        if nan_count == 0:
            total_correct += 1
        elif nan_count == 3:
            all_nan_count += 1
        else:
            partial_nan_count += 1

        # Replace NaN with an empty list
        step1 = step1 if isinstance(step1, list) else []
        step2 = step2 if isinstance(step2, list) else []
        step3 = step3 if isinstance(step3, list) else []

        # Combine steps and filter
        combined_steps = step1 + step2 + step3
        keep_items = [item for item, count in Counter(combined_steps).items() if count >= 2]
        all_keep_items.append(keep_items)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        all_keep_items.append([])

# Save the results to the DataFrame
CoreDF['specific_names'] = all_keep_items

# Print statistics
print(f'Added specific_names column to the DataFrame.')
print(f'Completely processed {total_correct} items correctly.')
print(f'{partial_nan_count} items had at least one NaN.')
print(f'{all_nan_count} items were entirely NaN.')
